<a href="https://colab.research.google.com/github/kusalsai/Smart-Tutor-AI-AI-Driven-Personalized-Teaching-Support/blob/kusal_ayinala/info5737_finalproject_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install llama-index


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [2]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import NodeParser # Import NodeParser
import tiktoken

In [3]:
!pip install --upgrade python-pptx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 12.5 MB/s eta 0:00:00


#Metadata


In [7]:
import os
import re


# Define the preprocessing function
def preprocess_text(file_path, text):
    """
    Preprocesses the text depending on the file type:
    - Removes punctuation for non-code files
    - Leaves code files untouched
    """
    # Define file extensions for code and text
    code_extensions = {".py", ".java", ".cpp", ".js", ".c", ".cs", ".html", ".css", ".php", ".rb"}
    text_extensions = {".pdf", ".docx", ".pptx", ".txt"}

    # Check the file extension
    ext = os.path.splitext(file_path)[-1].lower()

    # If it's a text file, remove punctuation
    if ext in text_extensions:
        # Remove punctuation and unnecessary whitespace
        text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Keep words and spaces only
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
        text = re.sub(r"\b\d{1,2}/\d{1,2}/\d{4}\b", "", text)  # Remove dates like 3/3/2025
        text = re.sub(r"\b\d{5,}\s\d+\b", "", text)  # Remove page numbers like 332025 54
    # If it's a code file (e.g., Python), leave the text as is
    elif ext in code_extensions:
        pass  # Do not modify code files

    return text



In [27]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceWindowNodeParser

# Assuming the PDF is directly in /content/
# If it's in a subfolder, adjust the path accordingly
document = SimpleDirectoryReader(
    input_files=["/content/Automatic Keyphrase Extraction A Survey of the State of the Art.pdf"]
).load_data()



In [28]:

# Apply metadata to each document and preprocess text
for doc in document:
    doc.metadata = {
        "file_name": doc.metadata.get("file_name", ""),
        "file_path": doc.metadata.get("file_path", ""),
        "num_tokens": len(doc.get_content().split()),  # Estimate token count
        "num_chars": len(doc.get_content()),  # Count characters
    }

    # Preprocess the document's text and update content using set_content()
    doc_text = doc.get_content()  # Use get_content() to retrieve text
    processed_text = preprocess_text(doc.metadata["file_path"], doc_text)
    doc.set_content(processed_text)  # Use set_content() to update text


In [30]:
import re

# Define chunk size and overlap
CHUNK_SIZE = 512
CHUNK_OVERLAP = 50  # Overlap between chunks
PAD_CHAR = " "      # Padding character

def exact_chunk_text(text, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    """
    Splits text into fixed-size chunks of exactly `chunk_size` characters,
    with optional overlap between chunks.
    """
    chunks = []
    start = 0
    text_len = len(text)

    # Process text in chunks of exact size
    while start < text_len:
        end = start + chunk_size
        chunk = text[start:end]

        # If the chunk is smaller than `chunk_size`, pad it to ensure consistency
        if len(chunk) < chunk_size:
            chunk = chunk.ljust(chunk_size, PAD_CHAR)

        chunks.append(chunk)

        # Move the starting point of the next chunk by `chunk_size - overlap` to ensure overlap
        start = end - overlap

    return chunks

class TextNode:
    def __init__(self, text, id_):
        self.text = text
        self.id_ = id_

# Process documents and create chunks with metadata
nodes = []  # Store TextNode-like dictionary objects

for doc in document:
    text = doc.get_content().title()  # Convert to Title Case
    text_chunks = exact_chunk_text(text)  # Exact chunking

    for chunk in text_chunks:
        metadata = {
            "file_name": doc.metadata.get("file_name", ""),
            "file_path": doc.metadata.get("file_path", ""),
            "num_tokens": len(chunk.split()),  # Estimate token count
            "num_chars": len(chunk),           # Number of characters
        }
        nodes.append({"doc_id": doc.doc_id, "text": chunk, "metadata": metadata})

# Print processed chunks for debugging
for i, doc in enumerate(nodes):
    print(f"\nChunk {i + 1}:")
    print(f"Document ID: {doc['doc_id']}")
    print(f"Metadata: {doc['metadata']}")
    print(f"Text: {doc['text'][:]}...")  # Show only the first 100 characters for brevity



Chunk 1:
Document ID: 2606f7ce-2f6a-40cb-b854-c9c58deaab62
Metadata: {'file_name': 'Automatic Keyphrase Extraction A Survey of the State of the Art.pdf', 'file_path': '/content/Automatic Keyphrase Extraction A Survey of the State of the Art.pdf', 'num_tokens': 66, 'num_chars': 512}
Text: Proceedings Of The 52Nd Annual Meeting Of The Association For Computational Linguistics Pages 12621273 Baltimore Maryland Usa June 2325 2014C2014 Association For Computational Linguistics Automatic Keyphrase Extraction A Survey Of The State Of The Art Kazi Saidul Hasanand Vincent Ng Human Language Technology Research Institute University Of Texas At Dallas Richardson Tx 750830688 Saidulvincehltutdallasedu Abstract While Automatic Keyphrase Extraction Has Been Examined Extensively Stateofthe Art Performance ...

Chunk 2:
Document ID: 2606f7ce-2f6a-40cb-b854-c9c58deaab62
Metadata: {'file_name': 'Automatic Keyphrase Extraction A Survey of the State of the Art.pdf', 'file_path': '/content/Automatic Keyphr

#storing all the data in chroma db

In [31]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 5.0 MB/s eta 0:0

In [32]:
!pip uninstall llama-index-vector-stores-chroma -y

In [33]:
# Install the necessary packages
!pip install --upgrade llama-index
!pip uninstall llama-index-vector-stores-chroma -y # uninstall the conflicting package

In [34]:
!pip install llama-index-vector-stores-chroma # Install the necessary package to add vector_stores to llama_index
# Install the necessary packages
!pip install --upgrade llama-index
!pip uninstall llama-index-vector-stores-chroma -y # uninstall the conflicting package

# Import necessary libraries


Found existing installation: llama-index-vector-stores-chroma 0.4.1
Uninstalling llama-index-vector-stores-chroma-0.4.1:
  Successfully uninstalled llama-index-vector-stores-chroma-0.4.1


In [35]:
!pip install llama-index-vector-stores-chroma
from typing import Collection
from pathlib import Path
from chromadb import PersistentClient
from chromadb.api.types import Documents, Embeddings, Metadatas, IDs
from llama_index.vector_stores.chroma import ChromaVectorStore  # Correct import



  Using cached llama_index_vector_stores_chroma-0.4.1-py3-none-any.whl.metadata (696 bytes)
Using cached llama_index_vector_stores_chroma-0.4.1-py3-none-any.whl (5.2 kB)


In [36]:

chroma_db = "./chroma_db"

# Ensure the directory exists
Path(chroma_db).mkdir(parents=True, exist_ok=True)

try:
    # Initialize ChromaDB
    chroma_client = PersistentClient(path=str(chroma_db))
    vector_store = ChromaVectorStore(chroma_client, collection_name="chunks")

    print("Chroma DB installed and initialized successfully")
except Exception as e:
    print(f"Chroma DB encountered an error: {e}")

Chroma DB installed and initialized successfully


In [37]:
# Install necessary packages
!pip install --upgrade llama-index
!pip uninstall llama-index-vector-stores-chroma -y # uninstall the conflicting package



Found existing installation: llama-index-vector-stores-chroma 0.4.1
Uninstalling llama-index-vector-stores-chroma-0.4.1:
  Successfully uninstalled llama-index-vector-stores-chroma-0.4.1


In [38]:
from llama_index.core.ingestion import IngestionPipeline  # Correct import for IngestionPipeline
from llama_index.core.node_parser.text import SentenceSplitter  # Correct import for SentenceSplitter
from llama_index.core.node_parser.text import TokenTextSplitter  # Correct import for TokenTextSplitter

# Ensure you have the vector_store object properly initialized
pipeline = IngestionPipeline(
    transformations=[
        TokenTextSplitter(chunk_size=1024),  # TokenTextSplitter now requires a chunk_size parameter
        SentenceSplitter(chunk_size=1024, chunk_overlap=0),  # Corrected parameter name
    ],
    vector_store=vector_store,
)

print("Ingestion pipeline initialized successfully!")


Ingestion pipeline initialized successfully!


In [39]:

try:
  nodes = pipeline.run(documents=document)
  if not nodes:
    print("no nodes were created")
    exit()
  print(f"{len(nodes)} document nodes created in chroma db")
except Exception as e:
  print(f"An error occurred: {e}")


14 document nodes created in chroma db


In [40]:
try:
  nodes = pipeline.run(documents=document)
  if not nodes:
    print("no nodes were created")
    exit()
  print(f"{len(nodes)} document nodes created in chroma db")

  # Print all nodes:
  for i, node in enumerate(nodes):
    print(f"Node {i + 1}:")
    print(f"  Text: {node.text}")  # Print the text content of the node
    print(f"  ID: {node.id_}")    # Print the unique ID of the node
    print("-" * 20)  # Add a separator for better readability

except Exception as e:
  print(f"An error occurred: {e}")

14 document nodes created in chroma db
Node 1:
  Text: Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics pages 12621273 Baltimore Maryland USA June 2325 2014c2014 Association for Computational Linguistics Automatic Keyphrase Extraction A Survey of the State of the Art Kazi Saidul Hasanand Vincent Ng Human Language Technology Research Institute University of Texas at Dallas Richardson TX 750830688 saidulvincehltutdallasedu Abstract While automatic keyphrase extraction has been examined extensively stateofthe art performance on this task is still much lower than that on many core natural lan guage processing tasks We present a sur vey of the state of the art in automatic keyphrase extraction examining the major sources of errors made by existing systems and discussing the challenges ahead 1 Introduction Automatic keyphrase extraction concerns the au tomatic selection of important and topical phrases from the body of a document Turney 2000 In other wo

#seach engine

In [52]:
!pip show llama_index


Name: llama-index
Version: 0.12.25
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 


In [54]:
!pip install --upgrade llama_index


In [56]:
from llama_index.core import VectorStoreIndex, ServiceContext  # Import from llama_index.core

In [62]:
# Example documents to be indexed (use your actual document data)
documents = [
    "Python indentation is crucial as it defines code blocks.",
    "In Python, indentation is used to define the structure of control flow.",
    "Python is known for its clean and readable syntax, partially due to indentation."
]

# Create the ServiceContext, using defaults
service_context = ServiceContext.from_defaults()

# Create the vector index from the documents
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

# Convert the index to a retriever
retriever = index.as_retriever()

# Define a custom query engine
class CustomQueryEngine(QueryEngine):
    def __init__(self, retriever: BaseRetriever):
        self.retriever = retriever
        self.synthesizer = get_response_synthesizer(response_mode="compact")

    def query(self, query_str: str):
        # Retrieve relevant nodes for the query
        nodes = self.retriever.retrieve(query_str)

        # Synthesize a response
        response = self.synthesizer.synthesize(query_str, nodes)

        return response

# Create the query engine instance
query_engine = CustomQueryEngine(retriever=retriever)

# Example search query
query = "What is Python indentation?"

# Run the query and get the response
response = query_engine.query(query)

# Output the response
print(response)


ValueError: ServiceContext is deprecated. Use llama_index.settings.Settings instead, or pass in modules to local functions/methods/interfaces.
See the docs for updated usage/migration: 
https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/